In [2]:
# streamlit run lseg_app.py -- --weights="/mnt/g/Datasets/ADEChallengeData2016/demo_e200.ckpt" --data-path="/mnt/g/Datasets/ADEChallengeData2016/"


# /home/yilu/miniconda3/envs/langseg/lib/python3.11/site-packages/encoding/nn/__init__.py

# https://github.com/zhanghang1989/PyTorch-Encoding/compare/master...johndavidbustard:PyTorch-Encoding:master
# tried:
# https://github.com/krrish94/PyTorch-Encoding/archive/refs/heads/master.zip
# 
# https://github.com/Zacchaeus00/PyTorch-Encoding/archive/refs/heads/master.zip
# from encoding import gpu

!python --version


/bin/bash: /opt/miniconda/envs/yz_langseg/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Python 3.11.4


In [9]:
# !python --version
import inspect
from pytorch_lightning import Trainer

# Extract the set of accepted arguments from the Trainer's signature
accepted_args = set(inspect.signature(Trainer).parameters.keys())
print(accepted_args)

trainer_args = {'num_nodes': 1, 'exp_name': 'lseg_ade20k_l16', 'dry_run': False, 'no_resume': False, 'accumulate_grad_batches': 2, 
                'max_epochs': 240, 'project_name': 'lightseg', 'data_path': '../datasets', 'dataset': 'ade20k', 'batch_size': 1, 
                'base_lr': 0.004, 'momentum': 0.9, 'weight_decay': 0.0001, 'aux': False, 'aux_weight': 0.2, 'se_loss': False, 'se_weight': 0.2, 
                'midasproto': False, 'ignore_index': -1, 'augment': False, 'backbone': 'clip_vitl16_384', 'num_features': 256, 'dropout': 0.1, 
                'finetune_weights': None, 'no_scaleinv': False, 'no_batchnorm': False, 'widehead': True, 'widehead_hr': False, 'arch_option': 0, 
                'block_depth': 0, 'activation': 'lrelu', 'gpus': -1, 'accelerator': 'ddp', 'benchmark': True, 'resume_from_checkpoint': None, 
                'version': 0, 'sync_batchnorm': True, 'callbacks': [], 'wandb_id': '9r6y3ocp', 'logger': []}

# Extract the set of keys from trainer_args
provided_args = set(trainer_args.keys())

# Find unexpected keywords
unexpected_args = provided_args - accepted_args

# Remove unexpected keywords from trainer_args
for arg in unexpected_args:
    trainer_args.pop(arg, None)

print(trainer_args)

{'gradient_clip_val', 'callbacks', 'overfit_batches', 'gradient_clip_algorithm', 'enable_checkpointing', 'min_epochs', 'check_val_every_n_epoch', 'plugins', 'benchmark', 'max_steps', 'fast_dev_run', 'use_distributed_sampler', 'detect_anomaly', 'barebones', 'reload_dataloaders_every_n_epochs', 'num_sanity_val_steps', 'sync_batchnorm', 'logger', 'max_time', 'limit_predict_batches', 'inference_mode', 'devices', 'num_nodes', 'max_epochs', 'log_every_n_steps', 'enable_model_summary', 'strategy', 'default_root_dir', 'val_check_interval', 'limit_test_batches', 'accelerator', 'precision', 'limit_val_batches', 'accumulate_grad_batches', 'enable_progress_bar', 'profiler', 'deterministic', 'min_steps', 'limit_train_batches'}
{'num_nodes': 1, 'accumulate_grad_batches': 2, 'max_epochs': 240, 'accelerator': 'ddp', 'benchmark': True, 'sync_batchnorm': True, 'callbacks': [], 'logger': []}


In [6]:
import copy

import itertools
import functools
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as torch_transforms
import encoding.datasets as enc_ds

encoding_datasets = {
    x: functools.partial(enc_ds.get_dataset, x)
    for x in ["coco", "ade20k", "pascal_voc", "pascal_aug", "pcontext", "citys"]
}


def get_dataset(name, **kwargs):
    if name in encoding_datasets:
        return encoding_datasets[name.lower()](**kwargs)
    assert False, f"dataset {name} not found"


print(list(encoding_datasets.keys()) )
get_dataset('ade20k')

['coco', 'ade20k', 'pascal_voc', 'pascal_aug', 'pcontext', 'citys']


NameError: name 'get_datase' is not defined

In [5]:
import os
labels = []
path = 'label_files/ade20k_objectInfo150.txt'
assert os.path.exists(path), '*** Error : {} not exist !!!'.format(path)
f = open(path, 'r') 
lines = f.readlines()      
for line in lines: 
    label = line.strip().split(',')[-1].split(';')[0]
    labels.append(label)
f.close()
labels = labels[1:]
print(labels)
print(len(labels))

['wall', 'building', 'sky', 'floor', 'tree', 'ceiling', 'road', 'bed', 'windowpane', 'grass', 'cabinet', 'sidewalk', 'person', 'earth', 'door', 'table', 'mountain', 'plant', 'curtain', 'chair', 'car', 'water', 'painting', 'sofa', 'shelf', 'house', 'sea', 'mirror', 'rug', 'field', 'armchair', 'seat', 'fence', 'desk', 'rock', 'wardrobe', 'lamp', 'bathtub', 'railing', 'cushion', 'base', 'box', 'column', 'signboard', 'chest', 'counter', 'sand', 'sink', 'skyscraper', 'fireplace', 'refrigerator', 'grandstand', 'path', 'stairs', 'runway', 'case', 'pool', 'pillow', 'screen', 'stairway', 'river', 'bridge', 'bookcase', 'blind', 'coffee', 'toilet', 'flower', 'book', 'hill', 'bench', 'countertop', 'stove', 'palm', 'kitchen', 'computer', 'swivel', 'boat', 'bar', 'arcade', 'hovel', 'bus', 'towel', 'light', 'truck', 'tower', 'chandelier', 'awning', 'streetlight', 'booth', 'television', 'airplane', 'dirt', 'apparel', 'pole', 'land', 'bannister', 'escalator', 'ottoman', 'bottle', 'buffet', 'poster', 's

In [4]:
import pathlib, os
from glob import glob
def get_latest_version(folder):
    versions = [
        int(pathlib.PurePath(path).name.split("_")[-1])
        for path in glob(f"{folder}/version_*/")
    ]

    if len(versions) == 0:
        return None

    versions.sort()
    return versions[-1]

version = get_latest_version(f"./checkpoints/lseg_ade20k_l16")
print(version)
exp_name = 'lseg_ade20k_l16'

def get_latest_checkpoint(exp_name, version):
    while version > -1:
        folder = f"./checkpoints/{exp_name}/version_{version}/checkpoints/"

        latest = f"{folder}/last.ckpt"
        if os.path.exists(latest):
            return latest, version

        chkpts = glob(f"{folder}/epoch=*.ckpt")

        if len(chkpts) > 0:
            break

        version -= 1

    if len(chkpts) == 0:
        return None, None

    latest = max(chkpts, key=os.path.getctime)

    return latest, version

latest, version = get_latest_checkpoint(exp_name, version)
print(latest, version)

0
./checkpoints/lseg_ade20k_l16/version_0/checkpoints//last.ckpt 0
